##Loading Libraries

In [1]:
!pip install -q timm
!pip install -q rasterio
!pip install -q geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 16.4 MB/s eta 0:00:00


In [2]:
import timm
from fastai.vision.all import *
from PIL import Image
from typing import Tuple
import rasterio as rio

##Prepare Example dataset

In [3]:
#Download the sample training dataset to setup the model
!gdown --id 1wzGJEL8Eur6Oobu-4JeNXnAbjI_Z5gwA

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1wzGJEL8Eur6Oobu-4JeNXnAbjI_Z5gwA
To: /content/S1DWdatasetTiny.zip
100% 14.1M/14.1M [00:00<00:00, 115MB/s] 


In [4]:
#Load datasets from the downloaded file
!unzip -q /content/S1DWdatasetTiny.zip

In [5]:
#define the folders
input_folder = '/content/S1DWdatasetTiny/images'
label_folder = '/content/S1DWdatasetTiny/labels'

## Process the images

In [6]:
#rename the labels to match the S1 images
import os

# Iterate over the files
for filename in os.listdir(input_folder):
    # If the filename contains 'S1'
    if 'S1' in filename:
        # Replace 'S1' with 'DW' in the filename
        new_filename = filename.replace('S1', 'DW')
        # Get full paths
        old_file_path = os.path.join(input_folder, filename)
        new_file_path = os.path.join(input_folder, new_filename)
        # Rename the file
        os.rename(old_file_path, new_file_path)


##Preparing Deep Learning Model


In [7]:
def open_geotiff(fn, chans=None):
    with rio.open(str(fn)) as f:
        data = f.read()
        data = data.astype(np.float32)
    im = torch.from_numpy(data).cpu()  # Ensure tensor is on CPU
    if chans is not None: im = im[chans]
    return im

# Custom class for handling multi-channel images
class MultiChannelTensorImage(TensorImage):
    _show_args = ArrayImageBase._show_args
    @classmethod
    def create(cls, fn, chans=None, **kwargs) ->None:
        return cls(open_geotiff(fn=fn, chans=chans))

    def __repr__(self): return f'{self.__class__.__name__} size={"x".join([str(d) for d in self.shape])}'


# Define a datablock for multi-channel images
def MultiChannelImageBlock(cls=MultiChannelTensorImage, chans=None):
    return TransformBlock(partial(cls.create, chans=chans))

# Custom DataLoader for segmentation tasks with multi-channel images
class TifSegmentationDataLoaders(DataLoaders):
    @classmethod
    @delegates(DataLoaders.from_dblock)
    def from_label_funcs(cls, path, fnames, label_func, chans=None, extensions=['.tif'], valid_pct=None, seed=None, codes=None, item_tfms=None, batch_tfms=None, **kwargs):
        dblock = DataBlock(blocks=(MultiChannelImageBlock(chans=chans), MaskBlock(codes=codes)),
                           splitter=RandomSplitter(valid_pct, seed=seed),
                           get_y=label_func,
                           item_tfms=item_tfms,
                           batch_tfms=batch_tfms)
        res = cls.from_dblock(dblock, fnames, path=path, **kwargs)
        return res

def get_mask_from_tif(fn):
    return open_geotiff(fn, chans=[0])[0]

# Combined loss function for segmentation
class CombinedLoss:
    def __init__(self, axis=1, smooth=1., alpha=1.):
        store_attr()
        self.focal_loss = FocalLossFlat(axis=axis)
        self.dice_loss = DiceLoss(axis, smooth)

    def __call__(self, pred, targ):
        return self.focal_loss(pred, targ) + self.alpha * self.dice_loss(pred, targ)

    def decodes(self, x):    return x.argmax(dim=self.axis)
    def activation(self, x): return F.softmax(x, dim=self.axis)

In [8]:
codes = ['other', 'water']; codes

['other', 'water']

In [9]:
batch_tfms = [Rotate(), Flip(), Dihedral(), Normalize()]

In [10]:
#setting up data loader, check the filepaths
dataLoaderS1Global = TifSegmentationDataLoaders.from_label_funcs(path=input_folder,
                                                   bs = 4,
                                                   codes=codes,
                                                   fnames = get_files(input_folder, extensions=['.tif'], recurse=False),
                                                   label_func = lambda o: get_mask_from_tif(f'/content/S1DWdatasetTiny/labels/{o.stem}{o.suffix}'),
                                                   valid_pct=0.2,
                                                   seed=42,
                                                   batch_tfms = batch_tfms)

In [11]:
#setup the deep learning model
modelS1Global =  unet_learner(dataLoaderS1Global, resnet34, metrics = [JaccardCoeff(), Dice()], loss_func=CombinedLoss(), opt_func=ranger, act_cls=Mish)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 209MB/s]


In [12]:
#Download the trained model
!gdown --id 18-9x4GwV52gUc7zztz1CPxvkmccajbWP

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=18-9x4GwV52gUc7zztz1CPxvkmccajbWP
From (redirected): https://drive.google.com/uc?id=18-9x4GwV52gUc7zztz1CPxvkmccajbWP&confirm=t&uuid=09995c47-a201-4762-8abd-c0df0f63ff1e
To: /content/model_resnet34_10epochs.pth
100% 660M/660M [00:11<00:00, 55.7MB/s]


In [13]:
#load the trained model
modelS1Global.load('/content/model_resnet34_10epochs')

##Inference

In [14]:
#download S1 images for inference
!gdown --id 1cQNgpxnKZUi0Ee2ozToU0XaHunY_7el0

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1cQNgpxnKZUi0Ee2ozToU0XaHunY_7el0
From (redirected): https://drive.google.com/uc?id=1cQNgpxnKZUi0Ee2ozToU0XaHunY_7el0&confirm=t&uuid=537eb9d2-ddf7-4ded-a691-7ba8d8018771
To: /content/downloadedS1_FZ90.zip
100% 175M/175M [00:00<00:00, 187MB/s]


In [15]:
#Load images from the downloaded file
!unzip -q /content/downloadedS1_FZ90.zip

In [16]:
#pre-processing the data after it has downloaded for inference
inputInference = '/content/downloadedS1_FZ90/'
outputInference = '/content/downloadedS1_FZ90_256/'

# Ensure the output folder exists
os.makedirs(outputInference, exist_ok=True)

In [17]:
#create patches of 256x256 for inference
import rasterio
from rasterio.windows import Window
import os
import glob
import logging
from concurrent.futures import ProcessPoolExecutor

# Set up logging
logging.basicConfig(filename='error_log.log', level=logging.ERROR, format='%(asctime)s:%(levelname)s:%(message)s')

def extract_and_save_patches(img_path, output_folder, patch_size=256):
    try:
        with rasterio.open(img_path) as dataset:
            width, height = dataset.width, dataset.height
            profile = dataset.profile

            x_start_points = list(range(0, width - patch_size, patch_size)) + [width - patch_size]
            y_start_points = list(range(0, height - patch_size, patch_size)) + [height - patch_size]

            for x in x_start_points:
                for y in y_start_points:
                    window = Window(x, y, patch_size, patch_size)
                    data = dataset.read(window=window)

                    profile.update({
                        'width': patch_size,
                        'height': patch_size,
                        'transform': rasterio.windows.transform(window, dataset.transform)
                    })

                    # Generate unique patch filename
                    base_name = os.path.splitext(os.path.basename(img_path))[0]
                    patch_filename = f"{output_folder}/{base_name}_{x}_{y}.tif"
                    with rasterio.open(patch_filename, 'w', **profile) as dst:
                        dst.write(data)
        return True
    except Exception as e:
        logging.error(f"Error processing {img_path}: {e}")
        with open('failed_files.txt', 'a') as f:
            f.write(f"{img_path}\n")
        return False

def process_image(args):
    img_path, output_folder, patch_size = args
    return extract_and_save_patches(img_path, output_folder, patch_size)

def process_folder(input_folder, output_folder, patch_size=256, max_workers=None):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    processed_images = set()
    images_to_process = [(img_path, output_folder, patch_size) for img_path in glob.glob(os.path.join(input_folder, '*.tif')) if os.path.splitext(os.path.basename(img_path))[0] not in processed_images]

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        results = executor.map(process_image, images_to_process)

    for img_path, success in zip(images_to_process, results):
        if success:
            # print(f"Patches for {img_path[0]} saved in '{output_folder}'.")
            processed_images.add(os.path.splitext(os.path.basename(img_path[0]))[0])
        else:
            print(f"Failed to process {img_path[0]}.")

# Protect the script's entry point
if __name__ == '__main__':
    input_folder = inputInference
    output_folder = outputInference
    process_folder(input_folder, output_folder)


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [20]:
#replace NaNs in S1 with 0s
import multiprocessing

def process_file(filename):
    # Construct the full file path
    file_path = os.path.join(outputInference, filename)
    try:
        with rasterio.open(file_path, 'r+') as src:
            # Read and process each band
            for i in range(1, src.count + 1):
                band = src.read(i)
                band[np.isnan(band)] = 0  # Replace NaNs with zeros
                src.write(band, i)  # Write the modified band back to the file
        # print(f"Processed {filename}")
    except Exception as e:
        print(f"Error processing {filename}: {str(e)}")

def main():
    # List all .tif files in the directory
    files = [f for f in os.listdir(outputInference) if os.path.splitext(f)[1].lower() == '.tif']
    # Create a pool of worker processes
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    # Map the process_file function to all files
    pool.map(process_file, files)
    # Close the pool and wait for all processes to complete
    pool.close()
    pool.join()

if __name__ == '__main__':
    main()


Processed SID69_2021-08-29_0_0_0.tif
Processed SID70_2021-08-30_-4_0_467.tif
Processed SID08_2022-07-27_2_423_256.tif
Processed SID71_2021-08-28_1_515_0.tif
Processed SID90_2022-06-12_2_97_97.tif
Processed SID47_2022-06-04_3_106_0.tifProcessed SID54_2022-06-11_3_69_0.tif

Processed SID02_2022-03-25_-2_0_0.tifProcessed SID83_2022-05-05_0_0_0.tif

Processed SID63_2022-06-07_0_0_0.tif
Processed SID02_2022-03-25_-2_0_100.tif
Processed SID72_2021-09-05_-1_0_447.tifProcessed SID05_2022-08-09_-1_256_256.tif

Processed SID11_2021-12-15_3_107_108.tif
Processed SID65_2022-05-26_2_0_0.tif
Processed SID24_2022-03-29_-4_128_127.tif
Processed SID89_2022-06-15_-1_0_0.tif
Processed SID62_2021-10-06_-9_0_372.tifProcessed SID32_2022-03-06_-2_56_56.tif

Processed SID71_2021-08-28_1_0_0.tif
Processed SID68_2022-05-18_1_193_192.tif
Processed SID56_2021-12-19_176_0_0.tif
Processed SID09_2021-06-18_-8_0_417.tifProcessed SID60_2022-06-11_3_199_0.tif

Processed SID12_2021-12-18_71_85_84.tif
Processed SID72_202

In [21]:
#setup the inference folder
inferSet = [fn for fn in sorted((Path(outputInference)).glob('**/*')) if fn.is_file()]

In [22]:
test_dl = modelS1Global.dls.test_dl(inferSet)

In [23]:
preds = modelS1Global.get_preds(dl=test_dl)

In [24]:
#save predicted images
import os
from PIL import Image
import numpy as np

# Create a folder named 'output_images' if it doesn't exist
output_folder = 'predictedImages'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Your loop for saving images
for i, pred in enumerate(preds[0]):
    pred_arg = pred.argmax(dim=0).numpy().astype(np.uint8)
    im = Image.fromarray(pred_arg)

    # Extract the image name from its full path
    image_name = str(inferSet[i].name).split('/')[-1]

    # Combine the folder name with the image name
    save_path = os.path.join(output_folder, image_name)

    # Save the image in the specified folder
    im.save(save_path)

print(f"All images have been saved in the '{output_folder}' folder.")

All images have been saved in the 'predictedImages' folder.


In [25]:
#georeference predictions

input_folder = 'predictedImages'
reference_folder = outputInference
output_folder = 'waterMapPredictions'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Iterate over all files in the input directory
for filename in os.listdir(input_folder):
    # Only process .tif files
    _, extension = os.path.splitext(filename)
    if extension.lower() != '.tif':
        continue

    input_filepath = os.path.join(input_folder, filename)
    reference_filepath = os.path.join(reference_folder, filename)

    # Read the reference image
    with rasterio.open(reference_filepath) as ref_src:
        ref_crs = ref_src.crs
        ref_transform = ref_src.transform

    # Read the original image
    with rasterio.open(input_filepath) as src:
        array = src.read()
        profile = src.profile

    # Update the profile with the reference CRS and transform
    profile.update({
        'crs': ref_crs,
        'transform': ref_transform,
    })

    # Write the image data back with the updated profile
    output_filepath = os.path.join(output_folder, filename)
    with rasterio.open(output_filepath, 'w', **profile) as dst:
        dst.write(array)


/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [26]:
#moaic the 256x256 patches to their original
from collections import defaultdict
import rasterio as rio
from rasterio.merge import merge

# Step 1: Scan folder and organize files into groups
input_folder = r"/content/waterMapPredictions"
output_folder = r"/content/waterMapPredictions_mosaic"

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

files = os.listdir(input_folder)
file_groups = defaultdict(list)

for file in files:
    if file.endswith('.tif'):
        parts = file.split("_")
        if len(parts) > 1:
            prefix = f"{parts[0]}_{parts[1]}"
            file_groups[prefix].append(os.path.join(input_folder, file))

# Step 2 and 3: Read files and mosaic them
for prefix, file_group in file_groups.items():
    src_files_to_mosaic = [rio.open(fp) for fp in file_group]

    # Merge function returns a single mosaic array and the transformation info
    mosaic, out_trans = merge(src_files_to_mosaic)

    # Copy the metadata
    out_meta = src_files_to_mosaic[0].meta.copy()

    # Update the metadata
    out_meta.update({"driver": "GTiff",
                     "height": mosaic.shape[1],
                     "width": mosaic.shape[2],
                     "transform": out_trans})

    # Step 4: Write the mosaic raster to disk
    with rio.open(os.path.join(output_folder, f"{prefix}.tif"), "w", **out_meta) as dest:
        dest.write(mosaic)

    # Close files
    for src in src_files_to_mosaic:
        src.close()


In [27]:
# prompt: zip a folder

!zip -r waterMapPredictions.zip /content/waterMapPredictions_mosaic/

  adding: content/waterMapPredictions_mosaic/ (stored 0%)
  adding: content/waterMapPredictions_mosaic/SID62_2021-10-06.tif (deflated 99%)
  adding: content/waterMapPredictions_mosaic/SID11_2021-12-15.tif (deflated 98%)
  adding: content/waterMapPredictions_mosaic/SID65_2022-05-26.tif (deflated 100%)
  adding: content/waterMapPredictions_mosaic/SID56_2021-12-19.tif (deflated 99%)
  adding: content/waterMapPredictions_mosaic/SID38_2022-01-28.tif (deflated 99%)
  adding: content/waterMapPredictions_mosaic/SID87_2022-05-12.tif (deflated 99%)
  adding: content/waterMapPredictions_mosaic/SID68_2022-05-18.tif (deflated 99%)
  adding: content/waterMapPredictions_mosaic/SID10_2022-03-27.tif (deflated 98%)
  adding: content/waterMapPredictions_mosaic/SID13_2022-03-14.tif (deflated 99%)
  adding: content/waterMapPredictions_mosaic/SID02_2022-03-25.tif (deflated 98%)
  adding: content/waterMapPredictions_mosaic/SID63_2022-06-07.tif (deflated 98%)
  adding: content/waterMapPredictions_mosaic/SID04